This file just converts the data from shards.txt and the rest of the low significance shards in the other file into a single csv file which is the one used by the rest ofthe code

In [77]:
from numpy import *
import pandas


params = ['vx','vy','vz','sigx','sigy','sigz','population']


# First 20 something
dat_old = loadtxt('../shards.txt', delimiter=',', usecols=arange(0,10))
names_old = loadtxt('../shards.txt', delimiter=',', usecols=10,dtype='str')
v_old = dat_old[:,0:3]
sig_old = dat_old[:,6:9]
pops_old = dat_old[:,9]

num_cands = size(names_old)
dat1 = zeros(shape=(num_cands,7))
for i in range(0,num_cands):
    dat1[i,0:3] = v_old[i,:]
    dat1[i,3:6] = sig_old[i,:]
    dat1[i,6] = 1.0*pops_old[i]
    
df_dat1 = pandas.DataFrame(dat1,columns=params)
df_names = pandas.DataFrame(names_old,columns=['ID'])
dat1_full = df_names.join(df_dat1)



df = pandas.read_csv('../GAIA_SDSS_substructures_next_30_cands.csv')
names = df.group_id.unique()


num_cands = size(names)
dat2 = zeros(shape=(num_cands,7))
for i in range(0,num_cands):
    Cand = df.loc[df['group_id'] == names[i]]
    U = Cand.GalRVel
    V = Cand.GalTVel
    W = Cand.GalzVel
    dat2[i,0:3] = array([mean(U),mean(V),mean(W)])
    dat2[i,3:6] = array([std(U),std(V),std(W)])
    dat2[i,6] = size(U)
    
df_dat2 = pandas.DataFrame(dat2,columns=params)
df_names = pandas.DataFrame(names,columns=['ID'])
dat2_full = df_names.join(df_dat2)

dat_full = dat1_full.append(dat2_full,ignore_index=True)
dat_full.to_csv('../Shards.csv',float_format='%.3f',index=False)

In [81]:
df = pandas.read_csv('../Shards.csv')
names = df.ID
nshards = size(names)
velocities = zeros(shape=(nshards,3))
dispersions = zeros(shape=(nshards,3))
velocities[0:(nshards),0] = df.vx
velocities[0:(nshards),1] = df.vy
velocities[0:(nshards),2] = df.vz
dispersions[0:(nshards),0] = df.sigx
dispersions[0:(nshards),1] = df.sigy
dispersions[0:(nshards),2] = df.sigz
pops = df.population
pops /= sum(pops)

In [92]:
print df.loc[[0]]

   ID   vx     vy    vz   sigx  sigy  sigz  population
0  S1 -8.6 -286.7 -67.9  115.3  49.9  60.0        34.0
